# Multi-Agent Crew with Tools - Fixed Version

This notebook includes error handling and alternative search tools in case Serper API has issues.

In [ ]:
from crewai import LLM
import os

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Verify API keys
print("API Keys Status:")
print(f"GEMINI_API_KEY: {'✅ Found' if os.getenv('GEMINI_API_KEY') else '❌ Missing'}")
print(f"SERPER_API_KEY: {'✅ Found' if os.getenv('SERPER_API_KEY') else '❌ Missing'}")

# Create a new LLM
llm = LLM(model="gemini/gemini-2.0-flash", temperature=0.2)
print("✅ LLM initialized")

In [ ]:
# Setup search tools with fallback options
from crewai_tools import SerperDevTool

def get_search_tool():
    """Get available search tool with fallback options"""
    
    # Try Serper first
    try:
        serper_tool = SerperDevTool()
        # Test the tool
        test_result = serper_tool.run("test")
        print("✅ SerperDevTool working")
        return serper_tool
    except Exception as e:
        print(f"❌ SerperDevTool failed: {e}")
    
    # Try DuckDuckGo as fallback
    try:
        from crewai_tools import DuckDuckGoSearchRun
        ddg_tool = DuckDuckGoSearchRun()
        print("✅ Using DuckDuckGo Search as fallback")
        return ddg_tool
    except ImportError:
        print("❌ DuckDuckGo tool not available")
    
    # Try Wikipedia as last resort
    try:
        from crewai_tools import WikipediaSearchTool
        wiki_tool = WikipediaSearchTool()
        print("✅ Using Wikipedia Search as fallback")
        return wiki_tool
    except ImportError:
        print("❌ Wikipedia tool not available")
        return None

# Get the best available search tool
search_tool = get_search_tool()

if not search_tool:
    print("⚠️ No search tools available. Agent will work with built-in knowledge only.")

In [ ]:
from crewai import Task, Agent, Crew

# Define your agents
researcher_agent = Agent(
    role='Research Specialist',
    goal='Research interesting facts about the topic: {topic}',
    backstory="You are an expert at finding relevant and factual data.",
    tools=[search_tool] if search_tool else [],
    verbose=True,
    llm=llm
)

writer_agent = Agent(
    role="Creative Writer",
    goal="Write a short blog summary using the research",
    backstory="You are skilled at writing engaging summaries based on provided content.",
    llm=llm,
    verbose=True,
)

print("✅ Agents created successfully")
print(f"Researcher tools: {len(researcher_agent.tools)} tool(s) available")

In [ ]:
# Create tasks
task1 = Task(
    description="Find 3-5 interesting and recent facts about {topic} as of year 2025.",
    expected_output="A bullet list of 3-5 facts",
    agent=researcher_agent,
)

task2 = Task(
    description="Write a 100-word blog post summary about {topic} using the facts from the research.",
    expected_output="A blog post summary",
    agent=writer_agent,
    context=[task1],
)

print("✅ Tasks created successfully")

In [ ]:
# Create and run the crew
crew = Crew(
    agents=[researcher_agent, writer_agent],
    tasks=[task1, task2],
    verbose=True,
)

print("🚀 Starting crew execution...")
result = crew.kickoff(inputs={"topic": "The future of electrical vehicles"})

print("\n" + "="*50)
print("FINAL RESULT:")
print("="*50)
print(result)